In [73]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import plotly.graph_objects as go

import random
import pandas as pd
import numpy as np

!pip install torchinfo
from torchinfo import summary

In [74]:
# Random Seed 고정 (학습 반복 시행 시에도 동일한 결과가 나오도록)

seed = 20250302

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

**1. 데이터셋 로딩 및 데이터 분석**

In [75]:
# 데이터셋 로딩

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transform,
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transform,
                                          download=True)


In [76]:
# 시간 절약을 위해, 학습 데이터에서 랜덤하게 일부 샘플만 추출

from torch.utils.data import Subset, DataLoader

NUM_TRAIN_SAMPLES = 8000
BATCH_SIZE = 32

subset_indices = random.sample(range(len(train_dataset)), NUM_TRAIN_SAMPLES)
train_subset = Subset(train_dataset, subset_indices)

train_loader = DataLoader(train_subset,
                          batch_size=BATCH_SIZE,
                          shuffle=True)

# 테스트 데이터셋은 학습 대상이 아니므로 그대로 이용
test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False)

In [77]:
# 클래스 불균형 분석

# 학습 데이터
train_labels = torch.tensor([train_subset.dataset.targets[i] for i in subset_indices])
train_class_counts = torch.bincount(train_labels)
print(train_class_counts)

NUM_CLASSES = len(train_class_counts)

tensor([721, 915, 793, 872, 778, 738, 795, 849, 770, 769])


In [78]:
train_class_percentage = np.array(train_class_counts) * 100.0 / sum(train_class_counts)

train_y_distrib = pd.DataFrame({'class': list(range(NUM_CLASSES)),
                                'count': train_class_counts,
                                'percentage (%)': train_class_percentage})

train_y_distrib

,class,count,percentage (%)
0,0,721,9.012500
1,1,915,11.437501
2,2,793,9.912500
3,3,872,10.900001
4,4,778,9.725000
5,5,738,9.225000
6,6,795,9.937500
7,7,849,10.612501
8,8,770,9.625000
9,9,769,9.612500


In [79]:
# 테스트 데이터
test_labels = test_loader.dataset.targets
test_class_counts = torch.bincount(test_labels)
print(test_class_counts)

tensor([ 980, 1135, 1032, 1010,  982,  892,  958, 1028,  974, 1009])


In [80]:
test_class_percentage = np.array(test_class_counts) * 100.0 / sum(test_class_counts)

test_y_distrib = pd.DataFrame({'class': list(range(NUM_CLASSES)),
                               'count': test_class_counts,
                               'percentage (%)': test_class_percentage})

test_y_distrib

,class,count,percentage (%)
0,0,980,9.80
1,1,1135,11.35
2,2,1032,10.32
3,3,1010,10.10
4,4,982,9.82
5,5,892,8.92
6,6,958,9.58
7,7,1028,10.28
8,8,974,9.74
9,9,1009,10.09


In [81]:
# CNN 모델 정의

class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        # Conv
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 128, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3),
            nn.ReLU()
        )
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3),
            nn.ReLU()
        )

        # Fully Connected
        self.fc1 = nn.Sequential(
            nn.Linear(512 * 4 * 4, 512),
            nn.Sigmoid()
        )
        self.fc_final = nn.Sequential(
            nn.Linear(512, 10),
            nn.Softmax()  # Classification Task 의 Output Layer 이므로 Softmax 고정
        )

    def forward(self, x):

        # Conv
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)

        x = x.view(-1, 512 * 4 * 4)

        # Fully Connected
        x = self.fc1(x)
        x = self.fc_final(x)

        return x

In [82]:
# 모델 구조 출력

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

print(summary(model, input_size=(BATCH_SIZE, 1, 28, 28)))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [32, 10]                  --
├─Sequential: 1-1                        [32, 128, 28, 28]         --
│    └─Conv2d: 2-1                       [32, 128, 28, 28]         1,280
│    └─ReLU: 2-2                         [32, 128, 28, 28]         --
├─MaxPool2d: 1-2                         [32, 128, 14, 14]         --
├─Sequential: 1-3                        [32, 256, 12, 12]         --
│    └─Conv2d: 2-3                       [32, 256, 12, 12]         295,168
│    └─ReLU: 2-4                         [32, 256, 12, 12]         --
├─MaxPool2d: 1-4                         [32, 256, 6, 6]           --
├─Sequential: 1-5                        [32, 512, 4, 4]           --
│    └─Conv2d: 2-5                       [32, 512, 4, 4]           1,180,160
│    └─ReLU: 2-6                         [32, 512, 4, 4]           --
├─Sequential: 1-6                        [32, 512]                 --


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1844: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return inner()


**3. 데이터셋 분리**

* Train Data -> Train Data + Valid Data (epoch) + Valid Data (하이퍼파라미터 최적화)

In [83]:
# 데이터셋 분리

from torch.utils.data import random_split

# 샘플 수
num_train = 1000
num_valid_epoch = 2000
num_valid_hpo = 5000

assert NUM_TRAIN_SAMPLES == num_train + num_valid_epoch + num_valid_hpo

# 데이터셋 분리
train_dataset, valid_epoch_dataset, valid_hpo_dataset =\
    random_split(train_subset, [num_train, num_valid_epoch, num_valid_hpo])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_epoch_loader = DataLoader(valid_epoch_dataset, batch_size=BATCH_SIZE, shuffle=False)
valid_hpo_loader = DataLoader(valid_hpo_dataset, batch_size=BATCH_SIZE, shuffle=False)

**4. 하이퍼파라미터 최적화 학습 실시 함수**

* 하이퍼파라미터 최적화 라이브러리는 Optuna 사용
* 하이퍼파라미터 탐색 100 회 실시
* 하이퍼파라미터 목록
  * Early Stopping 기준
    * Valid Data Accuracy
    * Valid Data Loss
  * Early Stopping 횟수
    * 3 ~ 20 범위의 자연수
  * Learning Rate
    * 0.0005 ~ 0.01 (= 5e-4 ~ 1e-2) 범위


In [84]:
MAX_EPOCHS = 65536
TRIAL_COUNT = 100  # HPO trial count

In [85]:
from sklearn.metrics import accuracy_score
from copy import deepcopy

In [86]:
# Optuna 설정

!pip install optuna
import optuna
import logging

optuna.logging.set_verbosity(logging.WARNING)

In [87]:
# 모델 학습 실시

# args :
# - model           : 학습할 모델
# - train_loader    : Training Data Loader
# - train_loss_list : 각 epoch 에서의 train loss 기록

# returns :
# - train_loss : 모델의 Train Loss

def run_train(model, train_loader, train_loss_list):
    model.train()
    train_loss = 0.0
    cnt = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # train 실시
        model.optimizer.zero_grad()
        outputs = model(images)

        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        model.optimizer.step()

        train_loss += loss.item()
        cnt += 1

    train_loss_list.append(train_loss / len(train_loader))
    return train_loss_list[-1]

In [88]:
# 모델 validation 실시

# args :
# - model        : validation 할 모델
# - valid_loader : Validation Data Loader
# - during_train : 모델 학습 중이면 True, 그렇지 않으면 False

# returns :
# - val_accuracy : 모델의 validation 정확도
# - val_loss     : 모델의 validation loss

def run_validation(model, valid_loader, during_train=True):
    model.eval()
    correct, total = 0, 0
    val_loss_sum = 0

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss_batch = nn.CrossEntropyLoss(reduction='sum')(outputs, labels)
            val_loss_sum += val_loss_batch

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Accuracy 계산
        val_accuracy = correct / total
        val_loss = val_loss_sum / total

    return val_accuracy, val_loss

In [89]:
# 모델 학습 및 validation 전체 프로세스

# args :
# - model              : 학습할 모델
# - train_loader       : Training Data Loader
# - valid_epoch_loader : 각 epoch 마다 validation 할 Valid Data Loader
# - valid_hpo_loader   : 최종적으로 해당 하이퍼파라미터 조합에 대한 Valid Data Loader
# - verbose            : 학습 중 프로세스 출력 여부

# returns :
# - final_acc        : 해당 하이퍼파라미터 조합에 대한 최종 Accuracy (valid accuracy 가 가장 높았던 epoch 의 모델로 측정)
# - best_epoch_model : valid accuracy 가 가장 높았던 epoch 에서 생성된 모델
# - epochs           : 해당 학습의 총 epoch count

def run_model_common(model, train_loader, valid_epoch_loader, valid_hpo_loader,
                     verbose=False):

    train_loss_list = []        # train loss
    valid_acc_list = []         # valid accuracy
    valid_loss_list = []        # valid loss

    max_valid_acc = 0.0         # max validation accuracy
    min_valid_loss = None       # min validation loss

    best_valid_acc_epoch = -1   # valid accuracy 가 가장 높았던 epoch
    best_valid_loss_epoch = -1  # valid loss 가 가장 낮았던 epoch
    best_epoch_model = None     # valid accuracy 가 가장 높았던 epoch 의 모델

    # 1. 학습 실시
    for epoch in range(MAX_EPOCHS):

        # 1-1. train model
        train_loss = run_train(model, train_loader, train_loss_list)

        # 1-2. validate model (with EPOCH VALID SET)
        epoch_acc, val_loss = run_validation(model, valid_epoch_loader)
        valid_acc_list.append(epoch_acc)
        valid_loss_list.append(val_loss)

        # 1-3. Early Stopping 처리 (overfitting 방지)
        if min_valid_loss is None or val_loss < min_valid_loss:

            # round 처리 이유: loss 의 미세한 개선의 반복으로 학습 과다 지연 방지
            min_valid_loss = round(float(val_loss), 4)
            best_valid_loss_epoch = epoch

        if epoch_acc > max_valid_acc:
            max_valid_acc = epoch_acc
            best_valid_acc_epoch = epoch

            best_epoch_model = CNN().to(device)
            best_epoch_model.load_state_dict(model.state_dict())

            if verbose:
                print('best model updated')

        # Early Stopping type 에 따른 학습 종료
        if model.early_stopping_type == 'val_acc':
            if epoch - best_valid_acc_epoch >= model.early_stopping_rounds:
                epochs = epoch
                break

        if model.early_stopping_type == 'val_loss':
            if epoch - best_valid_loss_epoch >= model.early_stopping_rounds:
                epochs = epoch
                break

        # 1-4. 결과 출력
        if verbose:
            print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Valid Loss: {val_loss:.4f}, Valid Accuracy: {epoch_acc:.4f}")

    # check best-epoch model correctly loaded
    checked_acc, _ = run_validation(best_epoch_model,
                                    valid_epoch_loader,
                                    during_train=False)

    if verbose:
        print(f"Best Epoch: {best_valid_acc_epoch}, Best Valid Acc: {max_valid_acc}")
        print(f"Valid Acc (with Epoch valid set) on Loaded Best Model: {checked_acc}")

    assert abs(max_valid_acc - checked_acc) < 1e-8

    # 2. validate best-epoch model (with HPO VALID SET)
    final_acc, _ = run_validation(best_epoch_model,
                                  valid_hpo_loader,
                                  during_train=False)

    if verbose:
        print(f"Final Acc (with HPO valid set) on Loaded Best Model: {final_acc}")

    return final_acc, best_epoch_model, epochs

In [90]:
print(device)

cuda


**4-1. 실험 실시**

In [91]:
hpo_best_acc = 0              # 모든 Hyper-param 조합의 HPO Valid set 정확도 중 가장 높은 것
best_hyperparam_set = None    # HPO Valid set 정확도가 가장 높은 Hyper-param 조합
best_hyperparam_model = None  # best_hyperparam_set 의 Hyper-param 조합으로 학습된 모델

In [92]:
trial_count = 0   # 1st ~ 10th trial 에만 학습 중 정보 출력
epoch_count = []  # 각 trial 의 epoch 횟수 리스트

def objective(trial):
    global hpo_best_acc, best_hyperparam_set, best_hyperparam_model, trial_count, epoch_count

    # hyper-params
    params = {
        'early_stopping_type': trial.suggest_categorical('early_stopping_type', ['val_acc', 'val_loss']),
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 3, 20),
        'learning_rate': trial.suggest_float('learning_rate', 0.0005, 0.01, log=True)
    }

    # define and run model
    model = CNN().to(device)
    model.optimizer = torch.optim.AdamW(model.parameters(),
                                        lr=params['learning_rate'])
    model.early_stopping_type = params['early_stopping_type']
    model.early_stopping_rounds = params['early_stopping_rounds']

    final_acc, best_epoch_model, epochs = run_model_common(model,
                                                           train_loader,
                                                           valid_epoch_loader,
                                                           valid_hpo_loader,
                                                           verbose=(trial_count < 10))

    trial_count += 1
    epoch_count.append(epochs)

    # global best model 갱신
    if final_acc > hpo_best_acc:
        hpo_best_acc = final_acc
        best_hyperparam_set = params

        best_hyperparam_model = CNN().to(device)
        best_hyperparam_model.load_state_dict(best_epoch_model.state_dict())

        print(f'best_hyperparam_model updated with Accuracy={hpo_best_acc:.4f}')

    print(f"Params: {params}, Accuracy: {final_acc:.4f}")
    return final_acc

In [93]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=TRIAL_COUNT)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


best model updated
Epoch 1, Train Loss: 2.2281, Valid Loss: 1.9834, Valid Accuracy: 0.5430
best model updated
Epoch 2, Train Loss: 1.8808, Valid Loss: 1.8773, Valid Accuracy: 0.5895
best model updated
Epoch 3, Train Loss: 1.8229, Valid Loss: 1.7755, Valid Accuracy: 0.6955
best model updated
Epoch 4, Train Loss: 1.7269, Valid Loss: 1.6967, Valid Accuracy: 0.7685
best model updated
Epoch 5, Train Loss: 1.6214, Valid Loss: 1.6246, Valid Accuracy: 0.8435
best model updated
Epoch 6, Train Loss: 1.5809, Valid Loss: 1.6180, Valid Accuracy: 0.8480
best model updated
Epoch 7, Train Loss: 1.5735, Valid Loss: 1.6111, Valid Accuracy: 0.8560
best model updated
Epoch 8, Train Loss: 1.5594, Valid Loss: 1.5865, Valid Accuracy: 0.8835
best model updated
Epoch 9, Train Loss: 1.5031, Valid Loss: 1.5301, Valid Accuracy: 0.9455
best model updated
Epoch 10, Train Loss: 1.4825, Valid Loss: 1.5182, Valid Accuracy: 0.9505
Epoch 11, Train Loss: 1.4783, Valid Loss: 1.5211, Valid Accuracy: 0.9460
best model updat

In [94]:
# Test Dataset 성능 평가

print(f'best hyper-param: {best_hyperparam_set}, best acc: {hpo_best_acc}')

best hyper-param: {'early_stopping_type': 'val_loss', 'early_stopping_rounds': 16, 'learning_rate': 0.001628974619081214}, best acc: 0.9688


In [95]:
# best_hyperparam_model 이 정상적으로 load 되었는지 최종 확인

checked_hpo_acc, _ = run_validation(best_hyperparam_model,
                                    valid_hpo_loader,
                                    during_train=False)

print(f"Valid Acc (with HPO valid set) on Best Hyper-param Model: {checked_hpo_acc}")

assert abs(hpo_best_acc - checked_hpo_acc) < 1e-8

Valid Acc (with HPO valid set) on Best Hyper-param Model: 0.9688


In [96]:
# 테스트셋에 대한 최종 정확도

hpo_final_acc, _ = run_validation(best_hyperparam_model,
                                  test_loader,
                                  during_train=False)

print(f'Final HPO Acc (with test set) : {hpo_final_acc}')

Final HPO Acc (with test set) : 0.9747


**5. HPO 성능 결과 확인**

In [97]:
from optuna.visualization import plot_optimization_history

In [98]:
# HPO 추이

fig = plot_optimization_history(study)
fig.update_layout(width=1000,
                  height=650,
                  yaxis_title='Accuracy (HPO valid set)')
fig.show()

In [101]:
fig.update_layout(yaxis=dict(range=[0.94, 0.97]))
fig.show()

**6. 각 Hyperparameter 값에 따른 성능 분포 확인**

In [149]:
# trial DataFrame 가져오기

trials_df = study.trials_dataframe()

In [150]:
trials_df

,number,value,datetime_start,datetime_complete,duration,params_early_stopping_rounds,params_early_stopping_type,params_learning_rate,state
0,0,0.9588,2025-03-02 14:09:17.993106,2025-03-02 14:09:46.134617,0 days 00:00:28.141511,3,val_acc,0.001156,COMPLETE
1,1,0.9588,2025-03-02 14:09:46.134831,2025-03-02 14:10:22.126557,0 days 00:00:35.991726,7,val_acc,0.001098,COMPLETE
2,2,0.1156,2025-03-02 14:10:22.126691,2025-03-02 14:10:33.989498,0 days 00:00:11.862807,9,val_acc,0.006383,COMPLETE
3,3,0.0940,2025-03-02 14:10:33.989623,2025-03-02 14:10:42.969402,0 days 00:00:08.979779,7,val_acc,0.009916,COMPLETE
4,4,0.9604,2025-03-02 14:10:42.969523,2025-03-02 14:11:43.737768,0 days 00:01:00.768245,10,val_loss,0.003003,COMPLETE
...,...,...,...,...,...,...,...,...,...
95,95,0.1156,2025-03-02 16:36:40.130554,2025-03-02 16:36:48.057126,0 days 00:00:07.926572,6,val_loss,0.003244,COMPLETE
96,96,0.9638,2025-03-02 16:36:48.057294,2025-03-02 16:38:32.177686,0 days 00:01:44.120392,18,val_loss,0.000857,COMPLETE
97,97,0.9586,2025-03-02 16:38:32.177856,2025-03-02 16:38:50.175848,0 days 00:00:17.997992,3,val_acc,0.000608,COMPLETE
98,98,0.9598,2025-03-02 16:38:50.175999,2025-03-02 16:39:51.572107,0 days 00:01:01.396108,16,val_loss,0.001300,COMPLETE


In [151]:
# epoch count 를 trial DataFrame 에 추가

print(list(epoch_count))

[18, 39, 11, 7, 68, 46, 16, 18, 101, 35, 328, 159, 218, 250, 117, 69, 48, 137, 269, 60, 111, 143, 33, 23, 24, 105, 46, 29, 54, 42, 30, 98, 84, 515, 32, 210, 5, 10, 19, 51, 60, 156, 96, 69, 428, 51, 117, 71, 57, 44, 32, 235, 128, 136, 24, 79, 17, 85, 32, 172, 109, 40, 100, 73, 172, 46, 50, 35, 46, 36, 190, 209, 189, 195, 379, 133, 171, 32, 120, 73, 37, 240, 132, 154, 72, 282, 261, 160, 97, 68, 51, 129, 24, 140, 410, 7, 125, 18, 71, 186]


In [152]:
trials_df['epoch_count'] = epoch_count

In [153]:
trials_df

,number,value,datetime_start,datetime_complete,duration,params_early_stopping_rounds,params_early_stopping_type,params_learning_rate,state,epoch_count
0,0,0.9588,2025-03-02 14:09:17.993106,2025-03-02 14:09:46.134617,0 days 00:00:28.141511,3,val_acc,0.001156,COMPLETE,18
1,1,0.9588,2025-03-02 14:09:46.134831,2025-03-02 14:10:22.126557,0 days 00:00:35.991726,7,val_acc,0.001098,COMPLETE,39
2,2,0.1156,2025-03-02 14:10:22.126691,2025-03-02 14:10:33.989498,0 days 00:00:11.862807,9,val_acc,0.006383,COMPLETE,11
3,3,0.0940,2025-03-02 14:10:33.989623,2025-03-02 14:10:42.969402,0 days 00:00:08.979779,7,val_acc,0.009916,COMPLETE,7
4,4,0.9604,2025-03-02 14:10:42.969523,2025-03-02 14:11:43.737768,0 days 00:01:00.768245,10,val_loss,0.003003,COMPLETE,68
...,...,...,...,...,...,...,...,...,...,...
95,95,0.1156,2025-03-02 16:36:40.130554,2025-03-02 16:36:48.057126,0 days 00:00:07.926572,6,val_loss,0.003244,COMPLETE,7
96,96,0.9638,2025-03-02 16:36:48.057294,2025-03-02 16:38:32.177686,0 days 00:01:44.120392,18,val_loss,0.000857,COMPLETE,125
97,97,0.9586,2025-03-02 16:38:32.177856,2025-03-02 16:38:50.175848,0 days 00:00:17.997992,3,val_acc,0.000608,COMPLETE,18
98,98,0.9598,2025-03-02 16:38:50.175999,2025-03-02 16:39:51.572107,0 days 00:01:01.396108,16,val_loss,0.001300,COMPLETE,71


In [154]:
# Early Stopping Type 별, Learning Rate 에 따른 Accuracy 분포

import plotly.express as px

fig = px.scatter(trials_df,
                 x="params_learning_rate",
                 y="value",
                 color="params_early_stopping_type",
                 title="Accuracy Distribution by Learning Rate")

fig.update_layout(width=1000, height=600,
                  yaxis_title='Accuracy')

fig.show()

In [155]:
fig.update_layout(xaxis=dict(range=[0.00045, 0.0035]),
                  yaxis=dict(range=[0.945, 0.97]))
fig.show()

In [156]:
# Early Stopping Type 별, Early Stopping epoch 횟수에 따른 Accuracy 분포

import plotly.express as px

fig = px.scatter(trials_df,
                 x="params_early_stopping_rounds",
                 y="value",
                 color="params_early_stopping_type",
                 title="Accuracy Distribution by Early Stopping Rounds")

fig.update_layout(width=1000, height=600,
                  yaxis_title='Accuracy')

fig.show()

In [157]:
fig.update_layout(yaxis=dict(range=[0.945, 0.97]))
fig.show()

In [158]:
# Early Stopping Type 별, Early Stopping epoch 횟수에 따른 Epoch Count 분포

import plotly.express as px

fig = px.scatter(trials_df,
                 x="params_early_stopping_rounds",
                 y="epoch_count",
                 color="params_early_stopping_type",
                 color_continuous_scale=['#F53', '#DAF', '#14A'],
                 title="Epoch Count Distribution by Early Stopping Rounds")

fig.update_layout(width=1000, height=600,
                  yaxis_title='Epochs')

fig.show()